In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyfaidx import Fasta

from tqdm.notebook import tqdm

In [2]:
data = pd.read_csv("data/ClinVar/SNV_variants.csv")
data

/tmp/ipykernel_226562/1922461105.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/ClinVar/SNV_variants.csv")


,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,RS# (dbSNP),PhenotypeList,Origin,OriginSimple,Assembly,VariationID,ChromosomeAccession,Chromosome,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
0,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,150829393,Galloway-Mowat syndrome 1,germline,germline,GRCh37,4,NC_000015.9,15,85342440,G,A
1,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,150829393,Galloway-Mowat syndrome 1,germline,germline,GRCh38,4,NC_000015.10,15,84799209,G,A
2,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,267606829,"Mitochondrial complex 1 deficiency, nuclear ty...",germline;unknown,germline,GRCh37,5,NC_000011.9,11,126145284,C,T
3,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,267606829,"Mitochondrial complex 1 deficiency, nuclear ty...",germline;unknown,germline,GRCh38,5,NC_000011.10,11,126275389,C,T
4,15045,single nucleotide variant,NM_017547.4(FOXRED1):c.1289A>G (p.Asn430Ser),55572,FOXRED1,HGNC:26927,Pathogenic,267606830,"Mitochondrial complex 1 deficiency, nuclear ty...",germline,germline,GRCh37,6,NC_000011.9,11,126147412,A,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2835686,1952238,single nucleotide variant,NM_001844.5(COL2A1):c.1502G>A (p.Gly501Glu),1280,COL2A1,HGNC:2200,Likely pathogenic,-1,Spondyloepiphyseal dysplasia congenita,de novo,germline,GRCh38,1895438,NC_000012.12,12,47986361,C,T
2835687,1952239,single nucleotide variant,NM_000263.4(NAGLU):c.1978A>G (p.Asn660Asp),4669,NAGLU,HGNC:7632,Likely pathogenic,-1,"Mucopolysaccharidosis, MPS-III-B",germline,germline,GRCh37,1895439,NC_000017.10,17,40696002,A,G
2835688,1952239,single nucleotide variant,NM_000263.4(NAGLU):c.1978A>G (p.Asn660Asp),4669,NAGLU,HGNC:7632,Likely pathogenic,-1,"Mucopolysaccharidosis, MPS-III-B",germline,germline,GRCh38,1895439,NC_000017.11,17,42543984,A,G
2835689,1952240,single nucleotide variant,NM_000435.3(NOTCH3):c.2903C>A (p.Ser968Ter),4854,NOTCH3,HGNC:7883,Likely pathogenic,-1,"Cerebral arteriopathy, autosomal dominant, wit...",paternal,germline,GRCh37,1895440,NC_000019.9,19,15291863,G,T


In [3]:
data["Chromosome"].unique()

array(['15', '11', '14', '6', '2', '20', '10', '19', '16', '22', '1', '7',
       '8', '9', '21', '5', '4', '17', '18', '3', '12', 'na', '13', 'MT',
       'Y', 'X', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 'Un'], dtype=object)

In [4]:
valid_chromosomes = [str(i) for i in range(1, 23)] +["X", "Y"]


In [5]:
buffer_size = 7


# Process hg19 variants

In [6]:
hg_19_genome = Fasta('data/reference_genomes/GRCh37.p13/GRCh37.p13.genome.fa')
hg_19_genome.keys()

odict_keys(['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY', 'chrM', 'GL877870.2', 'GL877872.1', 'GL383535.1', 'JH159133.1', 'KB663606.1', 'KB021647.1', 'KE332497.1', 'JH159131.1', 'GL949745.1', 'JH720447.1', 'GL582968.1', 'JH720446.1', 'JH591181.2', 'JH720443.2', 'JH159134.2', 'JH636052.4', 'JH636053.3', 'JH636054.1', 'JH636056.1', 'JH636058.1', 'JH636057.1', 'KE332505.1', 'JH720451.1', 'JH720452.1', 'JH720453.1', 'JH720454.3', 'JH159136.1', 'JH806587.1', 'JH806588.1', 'JH806589.1', 'JH806590.2', 'JH806591.1', 'JH806592.1', 'JH806593.1', 'JH806594.1', 'JH806595.1', 'JH806596.1', 'JH806597.1', 'JH720448.1', 'JH806598.1', 'JH806599.1', 'JH806600.2', 'JH806601.1', 'JH806602.1', 'JH806573.1', 'JH806574.2', 'JH806575.1', 'JH806580.1', 'JH806583.1', 'JH806584.1', 'JH806585.1', 'JH806603.1', 'JH159150.3', 'GL582969.1', 'JH806577.1', 'JH80

In [7]:
hg19_data = data[data["Assembly"]=="GRCh37"]
hg19_data

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,RS# (dbSNP),PhenotypeList,Origin,OriginSimple,Assembly,VariationID,ChromosomeAccession,Chromosome,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
0,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,150829393,Galloway-Mowat syndrome 1,germline,germline,GRCh37,4,NC_000015.9,15,85342440,G,A
2,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,267606829,"Mitochondrial complex 1 deficiency, nuclear ty...",germline;unknown,germline,GRCh37,5,NC_000011.9,11,126145284,C,T
4,15045,single nucleotide variant,NM_017547.4(FOXRED1):c.1289A>G (p.Asn430Ser),55572,FOXRED1,HGNC:26927,Pathogenic,267606830,"Mitochondrial complex 1 deficiency, nuclear ty...",germline,germline,GRCh37,6,NC_000011.9,11,126147412,A,G
6,15046,single nucleotide variant,NM_025152.3(NUBPL):c.166G>A (p.Gly56Arg),80224,NUBPL,HGNC:20278,Uncertain significance,200401432,not provided|Inborn genetic diseases|Mitochond...,germline;paternal,germline,GRCh37,214885,NC_000014.8,14,32031331,G,A
8,15048,single nucleotide variant,NM_000410.4(HFE):c.845G>A (p.Cys282Tyr),3077,HFE,HGNC:4886,Conflicting interpretations of pathogenicity; ...,1800562,Hemochromatosis type 1|Porphyria cutanea tarda...,germline;unknown,germline,GRCh37,9,NC_000006.11,6,26093141,G,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2835681,1952235,single nucleotide variant,NM_000152.5(GAA):c.844G>C (p.Asp282His),2548,GAA,HGNC:4065,Likely pathogenic,-1,"Glycogen storage disease, type II",germline,germline,GRCh37,1895435,NC_000017.10,17,78081507,G,C
2835683,1952237,single nucleotide variant,NM_018706.7(DHTKD1):c.2546A>T (p.Glu849Val),55526,DHTKD1,HGNC:23537,Uncertain significance,778723208,Charcot-Marie-Tooth disease axonal type 2Q,germline,germline,GRCh37,1895437,NC_000010.10,10,12160891,A,T
2835685,1952238,single nucleotide variant,NM_001844.5(COL2A1):c.1502G>A (p.Gly501Glu),1280,COL2A1,HGNC:2200,Likely pathogenic,-1,Spondyloepiphyseal dysplasia congenita,de novo,germline,GRCh37,1895438,NC_000012.11,12,48380144,C,T
2835687,1952239,single nucleotide variant,NM_000263.4(NAGLU):c.1978A>G (p.Asn660Asp),4669,NAGLU,HGNC:7632,Likely pathogenic,-1,"Mucopolysaccharidosis, MPS-III-B",germline,germline,GRCh37,1895439,NC_000017.10,17,40696002,A,G


In [8]:
hg19_data["Chromosome"].unique()

array(['15', '11', '14', '6', '2', '20', '10', '19', '16', '22', '1', '7',
       '8', '9', '21', '5', '4', '17', '18', '3', '12', '13', 'MT', 'Y',
       'X', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 'Un'], dtype=object)

In [33]:
hg19_processed_data = []
for i, row in tqdm(hg19_data.iterrows()):
    if row["Chromosome"] not in valid_chromosomes:
        continue
        
    ix = row["PositionVCF"]-1
    
    if not hg_19_genome["chr"+row["Chromosome"]][ix]==row.ReferenceAlleleVCF:
        continue
    # assert hg_19_genome["chr"+row["Chromosome"]][ix]==row.ReferenceAlleleVCF or hg_19_genome["chr"+row["Chromosome"]][ix]=="N"
    
    preceding_sequence = hg_19_genome["chr"+row["Chromosome"]][ix-buffer_size:ix]
    following_sequence = hg_19_genome["chr"+row["Chromosome"]][ix+1:ix+buffer_size+1]
    hg19_processed_data.append({"ReferenceSequence": str(preceding_sequence).upper()+row.ReferenceAlleleVCF+str(following_sequence).upper(), 
                           "MutatedSequence": str(preceding_sequence).upper()+row.AlternateAlleleVCF+str(following_sequence).upper(),
                          "ClinicalSignificance": row.ClinicalSignificance, "OriginSimple": row.OriginSimple, "GeneSymbol": row.GeneSymbol, "HGNC_ID": row.HGNC_ID, "PhenotypeList": row.PhenotypeList,
                           "RS# (dbSNP)": row["RS# (dbSNP)"]
                          })

0it [00:00, ?it/s]

In [34]:
hg19_processed_data = pd.DataFrame(hg19_processed_data)
hg19_processed_data

,ReferenceSequence,MutatedSequence,ClinicalSignificance,OriginSimple,GeneSymbol,HGNC_ID,PhenotypeList,RS# (dbSNP)
0,CACCTGCGGGTGAGT,CACCTGCAGGTGAGT,Uncertain significance,germline,ZNF592,HGNC:28986,Galloway-Mowat syndrome 1,150829393
1,AAAGGTCCAGTCCTT,AAAGGTCTAGTCCTT,Pathogenic,germline,FOXRED1,HGNC:26927,"Mitochondrial complex 1 deficiency, nuclear ty...",267606829
2,GTTGTCAACATGTAC,GTTGTCAGCATGTAC,Pathogenic,germline,FOXRED1,HGNC:26927,"Mitochondrial complex 1 deficiency, nuclear ty...",267606830
3,GTCCCGAGGACTTCC,GTCCCGAAGACTTCC,Uncertain significance,germline,NUBPL,HGNC:20278,not provided|Inborn genetic diseases|Mitochond...,200401432
4,TATACGTGCCAGGTG,TATACGTACCAGGTG,Conflicting interpretations of pathogenicity; ...,germline,HFE,HGNC:4886,Hemochromatosis type 1|Porphyria cutanea tarda...,1800562
...,...,...,...,...,...,...,...,...
1300086,GAACCGGGACCTTGC,GAACCGGCACCTTGC,Likely pathogenic,germline,GAA,HGNC:4065,"Glycogen storage disease, type II",-1
1300087,CAGCAAGAGATGAGC,CAGCAAGTGATGAGC,Uncertain significance,germline,DHTKD1,HGNC:23537,Charcot-Marie-Tooth disease axonal type 2Q,778723208
1300088,GATGGGCCCAACGCC,GATGGGCTCAACGCC,Likely pathogenic,germline,COL2A1,HGNC:2200,Spondyloepiphyseal dysplasia congenita,-1
1300089,CTATGCCAACAAGCA,CTATGCCGACAAGCA,Likely pathogenic,germline,NAGLU,HGNC:7632,"Mucopolysaccharidosis, MPS-III-B",-1


# Process GRCh38 variants

In [11]:
grch38_genome = Fasta('data/reference_genomes/GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.fna')
grch38_genome.keys()

odict_keys(['NC_000001.11', 'NT_187361.1', 'NT_187362.1', 'NT_187363.1', 'NT_187364.1', 'NT_187365.1', 'NT_187366.1', 'NT_187367.1', 'NT_187368.1', 'NT_187369.1', 'NC_000002.12', 'NT_187370.1', 'NT_187371.1', 'NC_000003.12', 'NT_167215.1', 'NC_000004.12', 'NT_113793.3', 'NC_000005.10', 'NT_113948.1', 'NC_000006.12', 'NC_000007.14', 'NC_000008.11', 'NC_000009.12', 'NT_187372.1', 'NT_187373.1', 'NT_187374.1', 'NT_187375.1', 'NC_000010.11', 'NC_000011.10', 'NC_000012.12', 'NC_000013.11', 'NC_000014.9', 'NT_113796.3', 'NT_167219.1', 'NT_187377.1', 'NT_113888.1', 'NT_187378.1', 'NT_187379.1', 'NT_187380.1', 'NT_187381.1', 'NC_000015.10', 'NT_187382.1', 'NC_000016.10', 'NT_187383.1', 'NC_000017.11', 'NT_113930.2', 'NT_187384.1', 'NT_187385.1', 'NC_000018.10', 'NC_000019.10', 'NC_000020.11', 'NC_000021.9', 'NC_000022.11', 'NT_187386.1', 'NT_187387.1', 'NT_187388.1', 'NT_187390.1', 'NT_187391.1', 'NT_187392.1', 'NT_187393.1', 'NT_187394.1', 'NC_000023.11', 'NC_000024.10', 'NT_187395.1', 'NT_18

In [12]:
grch38_chrom_mapping = {}
for chrom in valid_chromosomes:
    c = chrom
    if chrom=="X":
        c = "23"
    elif chrom=="Y":
        c = "24"
    ref = [k for k in grch38_genome.keys() if "NC_00" in k and "0"+c+"." in k][-1]
    grch38_chrom_mapping[chrom] = ref

grch38_chrom_mapping

{'1': 'NC_000001.11',
 '2': 'NC_000002.12',
 '3': 'NC_000003.12',
 '4': 'NC_000004.12',
 '5': 'NC_000005.10',
 '6': 'NC_000006.12',
 '7': 'NC_000007.14',
 '8': 'NC_000008.11',
 '9': 'NC_000009.12',
 '10': 'NC_000010.11',
 '11': 'NC_000011.10',
 '12': 'NC_000012.12',
 '13': 'NC_000013.11',
 '14': 'NC_000014.9',
 '15': 'NC_000015.10',
 '16': 'NC_000016.10',
 '17': 'NC_000017.11',
 '18': 'NC_000018.10',
 '19': 'NC_000019.10',
 '20': 'NC_000020.11',
 '21': 'NC_000021.9',
 '22': 'NC_000022.11',
 'X': 'NC_000023.11',
 'Y': 'NC_000024.10'}

In [13]:
grch38_data = data[data["Assembly"]=="GRCh38"]
grch38_data

,#AlleleID,Type,Name,GeneID,GeneSymbol,HGNC_ID,ClinicalSignificance,RS# (dbSNP),PhenotypeList,Origin,OriginSimple,Assembly,VariationID,ChromosomeAccession,Chromosome,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF
1,15043,single nucleotide variant,NM_014630.3(ZNF592):c.3136G>A (p.Gly1046Arg),9640,ZNF592,HGNC:28986,Uncertain significance,150829393,Galloway-Mowat syndrome 1,germline,germline,GRCh38,4,NC_000015.10,15,84799209,G,A
3,15044,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),55572,FOXRED1,HGNC:26927,Pathogenic,267606829,"Mitochondrial complex 1 deficiency, nuclear ty...",germline;unknown,germline,GRCh38,5,NC_000011.10,11,126275389,C,T
5,15045,single nucleotide variant,NM_017547.4(FOXRED1):c.1289A>G (p.Asn430Ser),55572,FOXRED1,HGNC:26927,Pathogenic,267606830,"Mitochondrial complex 1 deficiency, nuclear ty...",germline,germline,GRCh38,6,NC_000011.10,11,126277517,A,G
7,15046,single nucleotide variant,NM_025152.3(NUBPL):c.166G>A (p.Gly56Arg),80224,NUBPL,HGNC:20278,Uncertain significance,200401432,not provided|Inborn genetic diseases|Mitochond...,germline;paternal,germline,GRCh38,214885,NC_000014.9,14,31562125,G,A
9,15048,single nucleotide variant,NM_000410.4(HFE):c.845G>A (p.Cys282Tyr),3077,HFE,HGNC:4886,Conflicting interpretations of pathogenicity; ...,1800562,Hemochromatosis type 1|Porphyria cutanea tarda...,germline;unknown,germline,GRCh38,9,NC_000006.12,6,26092913,G,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2835682,1952235,single nucleotide variant,NM_000152.5(GAA):c.844G>C (p.Asp282His),2548,GAA,HGNC:4065,Likely pathogenic,-1,"Glycogen storage disease, type II",germline,germline,GRCh38,1895435,NC_000017.11,17,80107708,G,C
2835684,1952237,single nucleotide variant,NM_018706.7(DHTKD1):c.2546A>T (p.Glu849Val),55526,DHTKD1,HGNC:23537,Uncertain significance,778723208,Charcot-Marie-Tooth disease axonal type 2Q,germline,germline,GRCh38,1895437,NC_000010.11,10,12118892,A,T
2835686,1952238,single nucleotide variant,NM_001844.5(COL2A1):c.1502G>A (p.Gly501Glu),1280,COL2A1,HGNC:2200,Likely pathogenic,-1,Spondyloepiphyseal dysplasia congenita,de novo,germline,GRCh38,1895438,NC_000012.12,12,47986361,C,T
2835688,1952239,single nucleotide variant,NM_000263.4(NAGLU):c.1978A>G (p.Asn660Asp),4669,NAGLU,HGNC:7632,Likely pathogenic,-1,"Mucopolysaccharidosis, MPS-III-B",germline,germline,GRCh38,1895439,NC_000017.11,17,42543984,A,G


In [42]:
grch38_processed_data = []
for i, row in tqdm(grch38_data.iterrows()):
    if row["Chromosome"] not in valid_chromosomes:
        continue
        
        
    chrom = grch38_chrom_mapping[row["Chromosome"]]
    ix = row["PositionVCF"]-1
    if not grch38_genome[chrom][ix]==row.ReferenceAlleleVCF:
        continue

    preceding_sequence = grch38_genome[chrom][ix-buffer_size:ix]
    following_sequence = grch38_genome[chrom][ix+1:ix+buffer_size+1]
    grch38_processed_data.append({"ReferenceSequence": str(preceding_sequence).upper()+row.ReferenceAlleleVCF+str(following_sequence).upper(), 
                           "MutatedSequence": str(preceding_sequence).upper()+row.AlternateAlleleVCF+str(following_sequence).upper(),
                          "ClinicalSignificance": row.ClinicalSignificance, "OriginSimple": row.OriginSimple, "GeneSymbol": row.GeneSymbol, "HGNC_ID": row.HGNC_ID, "PhenotypeList": row.PhenotypeList,
                           "RS# (dbSNP)": row["RS# (dbSNP)"]
                          })
    #if i>30:
    # break

0it [00:00, ?it/s]

In [44]:
grch38_processed_data = pd.DataFrame(grch38_processed_data)
grch38_processed_data

,ReferenceSequence,MutatedSequence,ClinicalSignificance,OriginSimple,GeneSymbol,HGNC_ID,PhenotypeList,RS# (dbSNP)
0,CACCTGCGGGTGAGT,CACCTGCAGGTGAGT,Uncertain significance,germline,ZNF592,HGNC:28986,Galloway-Mowat syndrome 1,150829393
1,AAAGGTCCAGTCCTT,AAAGGTCTAGTCCTT,Pathogenic,germline,FOXRED1,HGNC:26927,"Mitochondrial complex 1 deficiency, nuclear ty...",267606829
2,GTTGTCAACATGTAC,GTTGTCAGCATGTAC,Pathogenic,germline,FOXRED1,HGNC:26927,"Mitochondrial complex 1 deficiency, nuclear ty...",267606830
3,GTCCCGAGGACTTCC,GTCCCGAAGACTTCC,Uncertain significance,germline,NUBPL,HGNC:20278,not provided|Inborn genetic diseases|Mitochond...,200401432
4,TATACGTGCCAGGTG,TATACGTACCAGGTG,Conflicting interpretations of pathogenicity; ...,germline,HFE,HGNC:4886,Hemochromatosis type 1|Porphyria cutanea tarda...,1800562
...,...,...,...,...,...,...,...,...
1207701,GAACCGGGACCTTGC,GAACCGGCACCTTGC,Likely pathogenic,germline,GAA,HGNC:4065,"Glycogen storage disease, type II",-1
1207702,CAGCAAGAGATGAGC,CAGCAAGTGATGAGC,Uncertain significance,germline,DHTKD1,HGNC:23537,Charcot-Marie-Tooth disease axonal type 2Q,778723208
1207703,GATGGGCCCAACGCC,GATGGGCTCAACGCC,Likely pathogenic,germline,COL2A1,HGNC:2200,Spondyloepiphyseal dysplasia congenita,-1
1207704,CTATGCCAACAAGCA,CTATGCCGACAAGCA,Likely pathogenic,germline,NAGLU,HGNC:7632,"Mucopolysaccharidosis, MPS-III-B",-1


In [45]:
row

#AlleleID                                                         1952240
Type                                            single nucleotide variant
Name                          NM_000435.3(NOTCH3):c.2903C>A (p.Ser968Ter)
GeneID                                                               4854
GeneSymbol                                                         NOTCH3
HGNC_ID                                                         HGNC:7883
ClinicalSignificance                                    Likely pathogenic
RS# (dbSNP)                                                            -1
PhenotypeList           Cerebral arteriopathy, autosomal dominant, wit...
Origin                                                           paternal
OriginSimple                                                     germline
Assembly                                                           GRCh38
VariationID                                                       1895440
ChromosomeAccession                   

In [46]:
clinvar_processed_data = pd.concat([hg19_processed_data, grch38_processed_data])

In [47]:
clinvar_processed_data.to_csv("processed_data/clinvar_processed_snvs.csv", index=False)

In [48]:
clinvar_processed_data["ClinicalSignificance"].value_counts()

Uncertain significance                          1033627
Likely benign                                    783749
Benign                                           275297
Pathogenic                                       115338
Conflicting interpretations of pathogenicity     114752
                                                 ...   
Benign/Likely benign; association                     2
Benign; protective                                    2
association; drug response; risk factor               2
Pathogenic; protective                                2
Likely benign; association                            1
Name: ClinicalSignificance, Length: 81, dtype: int64

In [49]:
clinvar_processed_data

,ReferenceSequence,MutatedSequence,ClinicalSignificance,OriginSimple,GeneSymbol,HGNC_ID,PhenotypeList,RS# (dbSNP)
0,CACCTGCGGGTGAGT,CACCTGCAGGTGAGT,Uncertain significance,germline,ZNF592,HGNC:28986,Galloway-Mowat syndrome 1,150829393
1,AAAGGTCCAGTCCTT,AAAGGTCTAGTCCTT,Pathogenic,germline,FOXRED1,HGNC:26927,"Mitochondrial complex 1 deficiency, nuclear ty...",267606829
2,GTTGTCAACATGTAC,GTTGTCAGCATGTAC,Pathogenic,germline,FOXRED1,HGNC:26927,"Mitochondrial complex 1 deficiency, nuclear ty...",267606830
3,GTCCCGAGGACTTCC,GTCCCGAAGACTTCC,Uncertain significance,germline,NUBPL,HGNC:20278,not provided|Inborn genetic diseases|Mitochond...,200401432
4,TATACGTGCCAGGTG,TATACGTACCAGGTG,Conflicting interpretations of pathogenicity; ...,germline,HFE,HGNC:4886,Hemochromatosis type 1|Porphyria cutanea tarda...,1800562
...,...,...,...,...,...,...,...,...
1207701,GAACCGGGACCTTGC,GAACCGGCACCTTGC,Likely pathogenic,germline,GAA,HGNC:4065,"Glycogen storage disease, type II",-1
1207702,CAGCAAGAGATGAGC,CAGCAAGTGATGAGC,Uncertain significance,germline,DHTKD1,HGNC:23537,Charcot-Marie-Tooth disease axonal type 2Q,778723208
1207703,GATGGGCCCAACGCC,GATGGGCTCAACGCC,Likely pathogenic,germline,COL2A1,HGNC:2200,Spondyloepiphyseal dysplasia congenita,-1
1207704,CTATGCCAACAAGCA,CTATGCCGACAAGCA,Likely pathogenic,germline,NAGLU,HGNC:7632,"Mucopolysaccharidosis, MPS-III-B",-1
